Installing dependencies

In [ ]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.0 MB/s eta 0:00:00


Importing libraries

In [ ]:
from google_play_scraper import app, search, Sort, reviews
import pandas as pd
import time

Get the AI apps' IDs

In [ ]:
def search_ai_apps(query="AI", n_results=10):
    """Search for AI apps on Google Play Store"""
    print(f"Searching for '{query}' apps...")
    search_results = search(
        query,
        lang="en",
        country="us",
        n_hits=n_results
    )
    return search_results

Get the reviews based on IDs

In [ ]:
def get_app_reviews(app_id, app_name, max_reviews=100):
    """Retrieve reviews for a specific app"""
    print(f"Fetching reviews for {app_name} ({app_id})...")
    try:
        # Use reviews() instead of reviews_all() to limit the number of reviews
        review_results, continuation_token = reviews(
            app_id,
            lang="en",
            country="us",
            sort=Sort.NEWEST,
            count=max_reviews  # This limits the number of reviews
        )

        # Print the structure of the first review for debugging
        if review_results:
            print(f"Review keys available: {list(review_results[0].keys())}")

        # Convert to a structured format with safer field access
        review_data = []
        for review in review_results:
            review_item = {
                'app_name': app_name,
                'reviewer': review.get('userName', 'Unknown'),
                'date': review.get('at', None),
                'score': review.get('score', None),
                'content': review.get('content', ''),
                # Use get() with a default value for potentially missing keys
                'thumbs_up': review.get('thumbsUp', 0)  # Default to 0 if missing
            }
            review_data.append(review_item)

        return review_data
    except Exception as e:
        print(f"Error retrieving reviews for {app_name}: {str(e)}")
        # Print a full traceback for debugging
        import traceback
        traceback.print_exc()
        return []

# Main workflow
- Searches for AI assistant apps
- Retrieves their details (developer, rating, and number of ratings)
- Collects user reviews
- Stores the reviews in a Pandas DataFrame
- Saves them as a CSV file
- Displays a sample of the collected reviews

In [ ]:
# Search for AI apps
ai_apps = search_ai_apps(query="AI", n_results=10)

all_reviews = []

# Get reviews for each app
for app_info in ai_apps:
    app_id = app_info['appId']
    app_name = app_info['title']

    # Get app details to show more info
    app_details = app(app_id)
    print(f"\nApp: {app_name}")
    print(f"Developer: {app_details['developer']}")
    print(f"Rating: {app_details['score']} ({app_details['ratings']} ratings)")

    # Get reviews
    app_reviews = get_app_reviews(app_id, app_name, max_reviews=100)
    all_reviews.extend(app_reviews)

    # Pause to avoid hitting rate limits
    time.sleep(1)

# Create a DataFrame and save to CSV
if all_reviews:
    df = pd.DataFrame(all_reviews)
    csv_filename = "ai_app_reviews.csv"
    df.to_csv(csv_filename, index=False)
    print(f"\nSaved {len(all_reviews)} reviews to {csv_filename}")

    # Show sample of reviews
    print("\nSample reviews:")
    print(df[['app_name', 'score', 'content']].head(3))
else:
    print("No reviews were collected.")

Searching for 'AI' apps...

App: PolyBuzz:formerly Poly.AI
Developer: CLOUD WHALE INTERACTIVE TECHNOLOGY LLC.
Rating: 4.162272 (418286 ratings)
Fetching reviews for PolyBuzz:formerly Poly.AI (ai.socialapps.speakmaster)...
Review keys available: ['reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion']

App: Talkie: Creative AI Community
Developer: SUBSUP
Rating: 4.556132 (500126 ratings)
Fetching reviews for Talkie: Creative AI Community (com.weaver.app.prod)...
Review keys available: ['reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion']

App: Chai: Chat AI Platform
Developer: Chai Research Corp.
Rating: 4.2897487 (341084 ratings)
Fetching reviews for Chai: Chat AI Platform (com.Beauchamp.Messenger.external)...
Review keys available: ['reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount